#### 1. Examine both Open Meteo and Tomorrow and decide which one you'd prefer to use. What drove your decision?
Documentation for Open Meteo seems easier - so far

#### 2. What is the URL to the documentation? (You don't use code for this one)

https://open-meteo.com/en/docs

#### 3. Make a request for the current weather where you are born, or somewhere you've lived.

In [161]:
import requests
import openmeteo_requests
import requests_cache
from retry_requests import retry
import pandas as pd
import numpy as np

In [165]:
### From the documentation: ###
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [166]:
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": -23.5804,
	"longitude": -46.5856,
	"current": ["temperature_2m", "apparent_temperature"]
}
responses = openmeteo.weather_api(url, params=params, verify=False)

data = responses[0]

In [167]:
current = data.Current()

current_temperature = current.Variables(0).Value()
apparent_temperature = current.Variables(1).Value()

print(f"Current temperature: {current_temperature:.2f}°C")
print(f"Apparent temperature: {apparent_temperature:.2f}°C")



Current temperature: 14.75°C
Apparent temperature: 15.13°C


#### 4. Print out the country this location is in.


No clue

#### 5. Print out the difference between the current temperature and how warm it feels. Use "It feels ___ degrees colder" or "It feels ___ degrees warmer," not negative numbers.


In [168]:
diff = apparent_temperature - current_temperature

if diff > 0:
    print(f'It feels {diff:.2f} degrees warmer')
elif diff < 0:
    print(f'It feels {abs(diff):.2f} degrees colder')
else:
    print('It feels like it is')

It feels 0.38 degrees warmer


#### 6. What's the current temperature at Heathrow International Airport? Use the airport's IATA code to search.

In [143]:
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 51.4712,
	"longitude": -0.4565,
	"current": ["temperature_2m", "apparent_temperature"]
}

responses = openmeteo.weather_api(url, params=params, verify=False)

data = responses[0]

In [144]:
current = data.Current()
current_temperature = current.Variables(0).Value()

print(f"Current temperature: {current_temperature:.2f}°C")


Current temperature: 13.30°C


#### 7. What URL would I use to request a 3-day forecast at Heathrow?


In [181]:
url = "https://api.open-meteo.com/v1/forecast" 
params = {
	"latitude": 51.4712,
	"longitude": -0.4565,
	"daily": ["temperature_2m_max", "temperature_2m_min"],
	"forecast_days": 3
}

responses = openmeteo.weather_api(url, params=params, verify=False)

data = responses[0]

#### 8. Print the date of each of the 3 days you're getting a forecast for.

In [182]:
daily = data.Daily()

In [183]:
temp_max = daily.Variables(0).ValuesAsNumpy()
temp_min = daily.Variables(1).ValuesAsNumpy()

In [184]:
### From documentation ###
daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

dates = daily_data['date']

In [185]:
for date in dates:
    print(f'{str(date.date())}')

2025-06-10
2025-06-11
2025-06-12


#### 9. Print the maximum temperature of each of the days.

In [186]:
### Hello, GPT :)

weather_data = []

for date, temp_max, temp_min in zip(dates, temp_max, temp_min):
    entry = {
        "date": str(date.date()),
        "temp_max": float(temp_max),
        "temp_min": float(temp_min)
    }
    weather_data.append(entry)

In [187]:
for weather in weather_data:
    date = weather['date']
    temp_max = weather['temp_max']
    print(f'{date}: {temp_max:.2f}°C')

2025-06-10: 22.28°C
2025-06-11: 24.18°C
2025-06-12: 22.57°C


#### 10. Print only the day with the highest maximum temperature.

In [188]:
max_temp_max = 0
day_max = 0

for weather in weather_data:
    date = weather['date']
    temp_max = weather['temp_max']
    if temp_max > max_temp_max:
        max_temp_max = temp_max
        day_max = date

print(f'Highest temperature ({max_temp_max:.2f}°C) will be on {day_max}')


Highest temperature (24.18°C) will be on 2025-06-11


#### 11. Did you find this easier or more difficult than using the weatherapi.com, and why? Which would you recommend to someone interesting in building a tool around weather information?

Harder!!! WeatherApi was a dict, and it was so much easier to visualize and iterate